# 1. 네이버 날씨 정보 스크래핑 하기

In [80]:
import requests
from bs4 import BeautifulSoup
import datetime

res = requests.get('https://search.naver.com/search.naver?query=날씨')
soup = BeautifulSoup(res.text,'lxml')

# 현재 위치
location = soup.find('h2', attrs={'class':'blind'}).text

# 1. 현재 날씨 및 온도
present_weather = soup.find('span', attrs={'class':'weather before_slash'}).text
ele =  soup.find('div', attrs={'class':'temperature_text'})
present_thermo = ele.find('strong').text

#2. 최고 최저 기온
lowest_thermo = soup.find('span', attrs={'class':'lowest'})
highest_thermo = soup.find('span', attrs={'class':'highest'})

#3. 미세먼지 확인
microdust = soup.find('span', attrs={'class':'txt'}).text

while True:
    print('=====================================')
    print('날씨 확인 프로그램')
    print('=====================================')
    print('접속시간:', datetime.datetime.now())
    print('접속위치:', location)
    print('---------------------------')
    print('1. 현재 날씨 및 온도 확인')
    print('2. 최저/최고 온도 확인')
    print('3. 미세먼지 확인')
    print('0. 종료')
    print('---------------------------')
    user = input('번호를 입력하세요.')

    if user == '0': break
    elif user == '1':
        print('=====================================')
        print('현재 날씨 확인')
        print('=====================================')
        print('---------------------------')
        print(present_weather, '/', present_thermo)
        print('---------------------------')
    elif user == '2':
        print('=====================================')
        print('최저/최고 기온 확인')
        print('=====================================')
        print('---------------------------')
        print(f'{list(lowest_thermo.children)[0].text}: {list(lowest_thermo.children)[1].text}')
        print(f'{list(highest_thermo.children)[0].text}: {list(highest_thermo.children)[1].text}')
        print('---------------------------')
    elif user == '3':
        print('=====================================')
        print('미세먼지 확인')
        print('=====================================')
        print('---------------------------')
        print(microdust)
        print('---------------------------')

날씨 확인 프로그램
접속시간: 2023-11-06 15:21:33.370824
접속위치: 마포구 합정동
---------------------------
1. 현재 날씨 및 온도 확인
2. 최저/최고 온도 확인
3. 미세먼지 확인
0. 종료
---------------------------


번호를 입력하세요. 2


최저/최고 기온 확인
---------------------------
최저기온: 16°
최고기온: 16°
---------------------------
날씨 확인 프로그램
접속시간: 2023-11-06 15:21:39.664139
접속위치: 마포구 합정동
---------------------------
1. 현재 날씨 및 온도 확인
2. 최저/최고 온도 확인
3. 미세먼지 확인
0. 종료
---------------------------


번호를 입력하세요. 3


미세먼지 확인
---------------------------
좋음
---------------------------
날씨 확인 프로그램
접속시간: 2023-11-06 15:21:42.243611
접속위치: 마포구 합정동
---------------------------
1. 현재 날씨 및 온도 확인
2. 최저/최고 온도 확인
3. 미세먼지 확인
0. 종료
---------------------------


번호를 입력하세요. 0


# 2. 네이버에서 bts이미지를 검색한 후 나온 결과 중 10개의 이미지를 저장하기

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

IMAGE_EXTRACT_NUM = 10 
SEARCH_KEYWORD = 'bts'

# 1. 네이버 홈페이지 요청
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=webdriver.ChromeOptions())
driver.get('https://www.naver.com/')

# 2. 검색필드에 'bts' 검색
ele = driver.find_element(By.ID, 'query')
ele.send_keys('bts')
ele.submit()

# 3. 이미지 검색 결과 선택
time.sleep(2)
driver.find_element(By.CSS_SELECTOR, '#lnb > div.lnb_group > div > div.lnb_nav_area._nav_area_root > div > div.api_flicking_wrap._conveyer_root > div:nth-child(1)').click()
time.sleep(2)

# 4. 최초 검색된 이미지 10개 저장
imgs = driver.find_elements(By.CSS_SELECTOR, '#main_pack > section.sc_new.sp_nimage._fe_image_viewer_prepend_target > div.api_subject_bx._fe_image_tab_list_root.ani_fadein > div > div > div.image_tile._fe_image_tab_grid > div > div > div > a')

img_cnt  = 0
img_src_list = []
for img in imgs:
    img.click()
    try:
        img_src = driver.find_element(By.XPATH, '//*[@id="main_pack"]/section[1]/div/div/div[1]/div[2]/div[1]/img').get_attribute('src')
    except: continue

    if not img_src.startswith('http'):
        continue
        
    img_cnt += 1
    print(f'[{img_cnt}].{img_src}')
    img_src_list.append(img_src)
    
    if img_cnt == IMAGE_EXTRACT_NUM: break

# 5. 이미지 저장하기
import os 
import requests

path = 'c:/Temp/' 

os.chdir(path)  
if not os.path.exists('bts_imgs'):   
    os.makedirs('bts_imgs')

file_no = 1
os.chdir(path+'bts_imgs') 
for url in img_src_list:
    extension = url.split('.')[-1]  
    ext = ''
    if extension in ['jpg','JPG','jpeg','JPEG','png','PNG','gif','GIF']: 
        ext = '.'+extension
    else: 
        ext = '.jpg'

    file_name = str(file_no)+'-'+SEARCH_KEYWORD+ext 

    file_no += 1
    res = requests.get(url)
    with open(file_name, 'wb') as f:
        f.write(res.content)     
        
print('저장완료')
driver.close()

[1].https://search.pstatic.net/common/?src=http%3A%2F%2Fimgnews.naver.net%2Fimage%2F586%2F2023%2F08%2F28%2F0000063530_001_20230828130903586.jpg&type=sc960_832
[2].https://search.pstatic.net/common/?src=http%3A%2F%2Fimgnews.naver.net%2Fimage%2F214%2F2023%2F09%2F20%2F0001301192_001_20230920185703428.jpg&type=sc960_832
[3].https://search.pstatic.net/sunny/?src=https%3A%2F%2Fi.pinimg.com%2Foriginals%2Fb2%2F18%2F2f%2Fb2182fd22a3a6931be39f6c7e6a77bbf.jpg&type=sc960_832
[4].https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyMzA1MzFfMjQ1%2FMDAxNjg1NTI5NTAyOTk2.__HDO8RTot5eEWf8Y3MIn-1fI-gvIM5bQd231oFTaDMg.HKQ5zPkUCTHcJq2QwXTX0P82Lst_uw1mQCrIDIZTCVog.PNG.insertkn%2Fimage.png&type=sc960_832
[5].https://search.pstatic.net/sunny/?src=https%3A%2F%2Fi.pinimg.com%2Foriginals%2Fef%2Ff0%2Ff7%2Feff0f718525bbbdc00992a691f1711b3.jpg&type=sc960_832
[6].https://search.pstatic.net/common/?src=http%3A%2F%2Fimgnews.naver.net%2Fimage%2F020%2F2023%2F02%2F11%2F0003479072_001_202302110301

# 3. 멜론 사이트 최신곡 검색

In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=webdriver.ChromeOptions())
driver.get('https://www.melon.com/new/index.htm')
soup = BeautifulSoup(driver.page_source, 'lxml')

# 웹페이지에서 제목명 태그, 가수명 태그, 앨범명 태그 가져오기 
song_chart = soup.find('tbody')
song_titles = song_chart.find_all('div', attrs={'class':'ellipsis rank01'})
song_artists = song_chart.find_all('div', attrs={'class':'ellipsis rank02'})
song_albums = song_chart.find_all('div', attrs={'class':'ellipsis rank03'})

title_list=[]
artist_list=[]
album_list=[]
for title in song_titles:
    title = title.find('a')
    title_list.append(title.text)
for artist in song_artists:
    artist_list.append(list(artist.children)[1].text)
for album in song_albums:
    album = album.find('a')
    album_list.append(album.text)

for i in range(10):
    print(f'[{i+1}] {title_list[i]} | {artist_list[i]} | {album_list[i]}')


[1] ESPRESSO MARTINI | 알리 (ALi) | ESPRESSO MARTINI
[2] Brachio | 다린 | Brachio
[3] 나쁜여자 | 김나희 | 나쁜여자
[4] Start To Shine (feat.개코) | 규빈 | Start To Shine (feat.개코)
[5] 왜 이래(Feat. SURAN(수란)) | HAAN | 한준한줌
[6] 딥하게 (DEEP IN LOVE) | 멋진녀석들 (GreatGuys) | 靑春記錄  第2話 : 深
[7] 너의 계절이 돌아올거야 | 로시 (Rothy) | 너의 계절이 돌아올거야
[8] 어디까지 간 거야 | Chan (찬) | 어디까지 간 거야
[9] 사랑하게 됐나 봐 | 안예슬 | 사랑하게 됐나 봐
[10] 키치죠지의 검은 고양이 | 박근홍 | 키치죠지의 검은 고양이


# 4. 대문자 시작 단어 찾기

In [75]:
import re

s = 'Edit the Expression & Text to see matches. Roll over matches or the expression for details. PCRE & JavaScript flavors of RegEx are supported. Validate your expression with Tests mode.'

p = re.compile('[A-Z]+[a-z]*[A-Z]*[a-z]*')
print(p.findall(s))

['Edit', 'Expression', 'Text', 'Roll', 'PCRE', 'JavaScript', 'RegEx', 'Validate', 'Tests']


# 5. 이메일 주소 추출하기

In [79]:
import re

s = 'Ryan has sent an invoice email to john.d@yahoo.com by using his email id ryan.arjun@gmail.com and he also shared a copy to his boss rosy.gray@amazon.co.uk on the cc part.'

p = re.compile('[a-zA-Z]+[.]?\w*[@][a-z]+[.][a-z]+[.]?[a-z]*')
print(p.findall(s))

['john.d@yahoo.com', 'ryan.arjun@gmail.com', 'rosy.gray@amazon.co.uk']
